# Skills Handbook Plugin Demo

This notebook demonstrates how to use the Skills Handbook plugin with the Portkey Python SDK.

The Skills Handbook plugin enhances AI responses by retrieving relevant positive and negative examples from a Qdrant vector database and injecting them as context before the LLM processes the request.

## Prerequisites

1. Portkey Gateway running at `http://localhost:8787`
2. Qdrant instance running at `http://localhost:6333` with collections populated
3. OpenAI API key for both LLM calls and embeddings

In [ ]:
# Install required packages
!pip install portkey-ai python-dotenv

In [1]:
import os
from portkey_ai import Portkey
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Get API key from environment
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GATEWAY_URL = "http://localhost:8787"
QDRANT_URL = "http://localhost:6333"

ModuleNotFoundError: No module named 'portkey_ai'

# Skills Handbook Plugin Demo

This notebook demonstrates how to use the Skills Handbook plugin with the Portkey Python SDK.

The Skills Handbook plugin enhances AI responses by retrieving relevant positive and negative examples from a Qdrant vector database and injecting them as context before the LLM processes the request.

## Key Feature: Dynamic Config Per Request
You can pass different plugin configurations on each request, giving you full control over when and how to use the memory/handbook features!

In [ ]:
# Configure the plugin
config = {
    "strategy": {"mode": "single"},
    "targets": [
        {
            "provider": "openai",
            "api_key": OPENAI_API_KEY
        }
    ],
    "input_guardrails": [
        {
            "id": "skills-handbook-retrieval",
            "type": "mutator",
            "skills-handbook.handbook": {
                "credentials": {
                    "endpoint": QDRANT_URL,
                    "apiKey": "",  # Empty for local Qdrant
                    "openaiApiKey": OPENAI_API_KEY
                },
                "positiveCollectionName": "skills-handbook-positive",
                "negativeCollectionName": "skills-handbook-negative",
                "topK": 3,
                "scoreThreshold": 0.5,
                "includePositive": True,
                "includeNegative": True
            }
        }
    ]
}

# Initialize Portkey client
portkey = Portkey(
    base_url=GATEWAY_URL,
    config=config
)

# Make a request about API authentication
response = portkey.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "How should I handle API authentication in my application?"
        }
    ],
    model="gpt-3.5-turbo",
    max_tokens=200
)

print("Response with Skills Handbook:")
print("=" * 80)
print(response.choices[0].message.content)
print("=" * 80)

## Example 2: Dynamic Config Per Request (Recommended!)

The most flexible way to use the plugin is to pass the config directly in each request. This lets you dynamically control when to use memory without creating multiple clients.

In [ ]:
# Initialize a single Portkey client - no config needed upfront!
portkey = Portkey(
    base_url=GATEWAY_URL,
    api_key=OPENAI_API_KEY
)

print("Example 2A: Request WITHOUT memory")
print("=" * 80)

# Request 1: No config = no memory/handbook
response1 = portkey.chat.completions.create(
    messages=[{"role": "user", "content": "How do I handle API authentication?"}],
    model="gpt-3.5-turbo",
    max_tokens=150
)

print(response1.choices[0].message.content)
print()

print("=" * 80)
print("Example 2B: Request WITH memory - pass config directly")
print("=" * 80)

# Request 2: Pass config = use memory/handbook
response2 = portkey.chat.completions.create(
    messages=[{"role": "user", "content": "How do I handle API authentication?"}],
    model="gpt-3.5-turbo",
    max_tokens=150,
    config={
        "input_guardrails": [{
            "id": "skills-handbook",
            "type": "mutator",
            "skills-handbook.handbook": {
                "credentials": {
                    "endpoint": QDRANT_URL,
                    "apiKey": "",
                    "openaiApiKey": OPENAI_API_KEY
                },
                "positiveCollectionName": "skills-handbook-positive",
                "negativeCollectionName": "skills-handbook-negative",
                "topK": 3,
                "scoreThreshold": 0.5,
                "includePositive": True,
                "includeNegative": True
            }
        }]
    }
)

print(response2.choices[0].message.content)
print()

print("=" * 80)
print("Example 2C: Another request with DIFFERENT memory settings")
print("=" * 80)

# Request 3: Different config = different behavior
response3 = portkey.chat.completions.create(
    messages=[{"role": "user", "content": "What API mistakes should I avoid?"}],
    model="gpt-3.5-turbo",
    max_tokens=150,
    config={
        "input_guardrails": [{
            "id": "skills-handbook",
            "type": "mutator",
            "skills-handbook.handbook": {
                "credentials": {
                    "endpoint": QDRANT_URL,
                    "apiKey": "",
                    "openaiApiKey": OPENAI_API_KEY
                },
                "topK": 5,  # More examples
                "scoreThreshold": 0.6,  # Higher threshold
                "includePositive": False,  # Only show negative examples
                "includeNegative": True
            }
        }]
    }
)

print(response3.choices[0].message.content)
print("=" * 80)

print("\n✅ All three requests used the SAME client, but with different configs!")

## Example 2: Comparison - With vs Without Plugin

Let's compare responses with and without the Skills Handbook plugin to see the difference.

In [ ]:
# Request WITHOUT the plugin
portkey_no_plugin = Portkey(
    base_url=GATEWAY_URL,
    config={
        "strategy": {"mode": "single"},
        "targets": [{"provider": "openai", "api_key": OPENAI_API_KEY}]
    }
)

question = "What should I avoid when making API calls?"

# Response without plugin
response_no_plugin = portkey_no_plugin.chat.completions.create(
    messages=[{"role": "user", "content": question}],
    model="gpt-3.5-turbo",
    max_tokens=200
)

# Response with plugin
response_with_plugin = portkey.chat.completions.create(
    messages=[{"role": "user", "content": question}],
    model="gpt-3.5-turbo",
    max_tokens=200
)

print("\n" + "="*80)
print("WITHOUT Skills Handbook Plugin:")
print("="*80)
print(response_no_plugin.choices[0].message.content)

print("\n" + "="*80)
print("WITH Skills Handbook Plugin:")
print("="*80)
print(response_with_plugin.choices[0].message.content)
print("="*80)

## Example 3: Different Topics

The plugin retrieves context based on semantic similarity. Let's test with different API-related questions.

In [ ]:
questions = [
    "How do I handle API rate limits?",
    "What's the best way to store API credentials?",
    "Should I use pagination when fetching data from APIs?",
    "How should I handle API errors?"
]

for i, question in enumerate(questions, 1):
    response = portkey.chat.completions.create(
        messages=[{"role": "user", "content": question}],
        model="gpt-3.5-turbo",
        max_tokens=150
    )
    
    print(f"\n{'='*80}")
    print(f"Question {i}: {question}")
    print(f"{'='*80}")
    print(response.choices[0].message.content)
    print()

## Example 4: Customizing Plugin Behavior

You can customize various aspects of the plugin:

In [ ]:
# Configuration with custom parameters
custom_config = {
    "strategy": {"mode": "single"},
    "targets": [{"provider": "openai", "api_key": OPENAI_API_KEY}],
    "input_guardrails": [
        {
            "id": "skills-handbook-custom",
            "type": "mutator",
            "skills-handbook.handbook": {
                "credentials": {
                    "endpoint": QDRANT_URL,
                    "apiKey": "",
                    "openaiApiKey": OPENAI_API_KEY
                },
                "positiveCollectionName": "skills-handbook-positive",
                "negativeCollectionName": "skills-handbook-negative",
                "topK": 5,  # Retrieve more examples
                "scoreThreshold": 0.6,  # Higher threshold for more relevant results
                "includePositive": True,
                "includeNegative": False,  # Only positive examples
                "positivePrefix": "\n## Best Practices:\n",
                "positiveSuffix": "\n---\n"
            }
        }
    ]
}

portkey_custom = Portkey(
    base_url=GATEWAY_URL,
    config=custom_config
)

response = portkey_custom.chat.completions.create(
    messages=[{"role": "user", "content": "What are API best practices?"}],
    model="gpt-3.5-turbo",
    max_tokens=250
)

print("Response with customized plugin settings:")
print("="*80)
print(response.choices[0].message.content)
print("="*80)

## Example 5: Only Negative Examples

Sometimes you might want to focus only on what NOT to do:

In [ ]:
# Configuration with only negative examples
negative_only_config = {
    "strategy": {"mode": "single"},
    "targets": [{"provider": "openai", "api_key": OPENAI_API_KEY}],
    "input_guardrails": [
        {
            "id": "skills-handbook-negative-only",
            "type": "mutator",
            "skills-handbook.handbook": {
                "credentials": {
                    "endpoint": QDRANT_URL,
                    "apiKey": "",
                    "openaiApiKey": OPENAI_API_KEY
                },
                "positiveCollectionName": "skills-handbook-positive",
                "negativeCollectionName": "skills-handbook-negative",
                "topK": 3,
                "scoreThreshold": 0.5,
                "includePositive": False,  # Disable positive examples
                "includeNegative": True    # Only negative examples
            }
        }
    ]
}

portkey_negative = Portkey(
    base_url=GATEWAY_URL,
    config=negative_only_config
)

response = portkey_negative.chat.completions.create(
    messages=[{"role": "user", "content": "What mistakes should I avoid when working with APIs?"}],
    model="gpt-3.5-turbo",
    max_tokens=200
)

print("Response with only negative examples:")
print("="*80)
print(response.choices[0].message.content)
print("="*80)

## Example 6: Streaming Responses

The plugin also works with streaming responses:

In [ ]:
print("Streaming response with Skills Handbook:")
print("="*80)

stream = portkey.chat.completions.create(
    messages=[{"role": "user", "content": "Give me tips for API security"}],
    model="gpt-3.5-turbo",
    max_tokens=200,
    stream=True
)

for chunk in stream:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)

print("\n" + "="*80)

## Summary

The Skills Handbook plugin:

✅ **Enhances AI responses** with relevant examples from your knowledge base

✅ **Uses semantic search** via OpenAI embeddings and Qdrant

✅ **Supports both positive and negative examples** to guide behavior

✅ **Fully customizable** - adjust retrieval count, thresholds, formatting, etc.

✅ **Works with streaming** and all OpenAI-compatible models

### Use Cases:
- **Team Knowledge Sharing**: Store best practices that guide AI responses
- **Guardrails**: Prevent common mistakes by including negative examples
- **Domain Expertise**: Inject specialized knowledge for specific tools/domains
- **Consistent Behavior**: Ensure AI agents follow organizational standards